In [201]:
# Import statements
import urllib
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen
from dateutil.parser import parse
from collections import defaultdict

# Contents:
* [Get URLs](#Get-URLs)
    * [Climate change](#Climate-change)
        * [URLs from search](#URLs-from-search)
        * [MediaCloud URLs](#MediaCloud-urls)
        * [Reid Google search URLs](#Reid-Urls)
    * [Vaccines](#Vaccines)
* [Post-processing](#Post-processing)
    * [Get missing meta info](#Getting-missing-meta-info)
    * [Check consistency of coding](#Check-consistency-of-coding)
    * [Deduplicate via title similarity](#Dedup-via-title-similarity)
* [Scraping fulltext](#Scraping-fulltext)
* [Summary stats](#Summary-stats)

# Get URLs

In [58]:
SERP_API_KEY = "481df24348cbec5d00f65baa55986d30b3b1ef2b09c5ab9de0f667dd43ce51d2"
from serpapi.google_search_results import GoogleSearchResults

In [60]:
query_params = {"location":"United States", "device":"desktop", "hl":"en", "gl":"us", "serp_api_key":SERP_API_KEY}
client = GoogleSearchResults(query_params)

In [ ]:
def do_serpapi(domain,keyword):
    keyword = keyword.replace('_',' ').replace('+',' ').replace('-',' ')
    client.params_dict["q"] = "site:{} {}".format(domain,keyword)
    print('Searching w/ query: {}...'.format(client.params_dict["q"]))
    page_no = 1
    client.params_dict["start"] = (page_no-1)*10
    
    dict_list = []
    while 'error' not in client.get_dict():
        dict_list.append(client.get_dict())
        page_no += 1
        client.params_dict["start"] = (page_no-1)*10
    
    return dict_list

In [198]:
def parse_serpapi_results(d_list):
    meta = []
    for d in d_list:
        if 'error' in d:
            print(d['error'])
        elif d['search_metadata']['status'] == 'Success':
            res = d['organic_results']
            if d['search_information']['total_results'] <= 10:
                page_no = 1
            else:
                page_no = d['pagination']['current']
            print('Number of results on page {}: {}'.format(page_no,len(res)))
            meta.extend([(x['title'],x['link'],x['date']) if 'date' in x
                        else (x['title'],x['link']) for x in res])
        else:
            print("API get failure")
    return meta

## Climate change

In [107]:
KEYWORDS = ['climate_change','global_warming','fossil_fuels']

### URLs from search

In [202]:
URLS_PER_DOMAIN = defaultdict(dict)

In [108]:
DOMAINS = ['www.foxnews.com','www.breitbart.com','www.theblaze.com','www.pjmedia.com','www.nationalreview.com',
           'www.thenation.com','www.buzzfeednews.com','www.vox.com','www.washingtonpost.com','www.progressive.org',
          'www.nytimes.com','www.motherjones.com','www.democracynow.org']

In [ ]:
for DOMAIN in DOMAINS:
    for KW in KEYWORDS:
        dl = do_serpapi(DOMAIN,KW)
        results = parse_serpapi_results(dl)
        URLS_PER_DOMAIN[DOMAIN][KW] = results

Searching w/ query: site:www.foxnews.com climate change
Number of results on page 1: 10
Number of results on page 2: 10
Number of results on page 3: 10
Number of results on page 4: 10
Number of results on page 5: 10
Number of results on page 6: 10
Number of results on page 7: 10
Number of results on page 8: 10
Number of results on page 9: 10
Number of results on page 10: 10
Number of results on page 11: 10
Number of results on page 12: 10
Number of results on page 13: 10
Number of results on page 14: 10
Number of results on page 15: 10
Number of results on page 16: 10
Number of results on page 17: 10
Number of results on page 18: 10
Number of results on page 19: 10
Number of results on page 20: 10
Number of results on page 21: 10
Number of results on page 22: 10
Number of results on page 23: 10
Number of results on page 24: 10
Number of results on page 25: 10
Number of results on page 26: 10
Number of results on page 27: 10
Number of results on page 28: 10
Number of results on page 29:

In [ ]:
pickle.dump(URLS_PER_DOMAIN,open('google_search_res_climate_change.pkl','wb'))

### MediaCloud URLs

### Reid URLs

In [ ]:
# Just load them

## Vaccines

# Post-processing

## Getting missing meta info

## Check consistency of coding

In [ ]:
# Assertions

## Dedup via title similarity

In [ ]:
def d_l_dist(s1, s2):
    d = {}
    lenstr1 = len(s1)
    lenstr2 = len(s2)
    for i in range(-1,lenstr1+1):
        d[(i,-1)] = i+1
    for j in range(-1,lenstr2+1):
        d[(-1,j)] = j+1

    for i in range(lenstr1):
        for j in range(lenstr2):
            if s1[i] == s2[j]:
                cost = 0
            else:
                cost = 1
            d[(i,j)] = min(
                           d[(i-1,j)] + 1, # deletion
                           d[(i,j-1)] + 1, # insertion
                           d[(i-1,j-1)] + cost, # substitution
                          )
            if i and j and s1[i]==s2[j-1] and s1[i-1] == s2[j]:
                d[(i,j)] = min (d[(i,j)], d[i-2,j-2] + cost) # transposition

    return d[lenstr1-1,lenstr2-1]

def is_same(u1,u2):
    #Djk ≤ 0.2 × Min.[|Tj|,|Tk|]
    # determine Damerau-Levensthtein edit distance
    D_jk = d_l_dist(u1,u2)
    t_j = len(u1)
    t_k = len(u2)
    min_ = min(t_j,t_k)
    return D_jk < 0.2*min_ 

In [ ]:
outlet_groups = df_all.groupby('media_name')
outlet_groups.first()

# Scraping fulltext

# Summary stats

In [ ]:
# exclude articles w/ empty fulltext in final df